In [24]:
%matplotlib inline
import pandas as pd

In [22]:
%%html
<style>
.rendered_html p {
    font-size: 16px;
    font-family: InputMonoCondensed;
    text-align:justify}
</style>

IDs of customers who made at least 3 orders in both 2020 and 2021.

**Data View**  
id: int64  
user_id: object  
order_date: datetime64[ns]  
order_total: float64

**Approach**
pd.merge(df1, df2, how="inner", on= "id")

In [ ]:
(
    pd.merge(
        amazon_orders.query(
            'order_date >= "2021-01-01 00:00:00" and order_date <= "2021-12-31 11:59:59"'
        )
        .groupby(["user_id"])["id"]
        .count()
        .gt(2)
        .reset_index(),
        amazon_orders.query(
            'order_date >= "2020-01-01 00:00:00" and order_date <= "2020-12-31 11:59:59"'
        )
        .groupby(["user_id"])["id"]
        .count()
        .gt(2)
        .reset_index(),
        how="inner",
        on="user_id",
    ).query("id_x == True and id_y == True")
)["user_id"]


The election is conducted in a city and everyone can vote for one or more candidates, or choose not to vote at all. Each person has 1 vote so if they vote for multiple candidates, their vote gets equally split across these candidates. For example, if a person votes for 2 candidates, these candidates receive an equivalent of 0.5 vote each.

Find out who got the most votes and won the election. Output the name of the candidate or multiple names in case of a tie. To avoid issues with a floating-point error you can round the number of votes received by a candidate to 3 decimal places.

**Data View**  
voter: object  
candidate: object  


In [ ]:
(
    pd.merge(
        voting_results.dropna()
        .groupby("voter")
        .size()
        .to_frame("count")
        .rdiv(1)
        .reset_index(),
        voting_results.dropna(),
        how="left",
        on="voter",
    )
    .drop("voter", axis=1)
    .groupby("candidate")["count"]
    .sum()
    .reset_index()
    .nlargest(1, "count")
)


**Problem Statement**  
Find the customer with the highest daily total order cost between 2019-02-01 to 2019-05-01. If customer had more than one order on a certain day, sum the order costs on daily basis. Output customer's first name, total cost of their items, and the date.

Assumption: every first name in the dataset is unique.

**Data View**

Dataframe #1: customers  
id: int64  
first_name: object  
last_name: object  
city: object  
address: object  
phone_number: object

Dataframe #2: orders  
id: int64  
cust_id: int64  
order_date: datetime64[ns]  
order_details: object  
total_order_cost: int64


In [ ]:
(
    pd.merge(
        customers[["id", "first_name"]],
        (
            orders.query(
                'order_date > "2019-02-01 00:00:00" and order_date < "2019-05-01 11:59:59"'
            )
            .groupby(["cust_id", "order_date"])["total_order_cost"]
            .sum()
            .reset_index()
            .nlargest(1, "total_order_cost")
            .rename(columns={"cust_id": "id"})
        ),
        on="id",
        how="right",
    ).drop("id", axis=1)
)


American Express is reviewing their customers' transactions, and you have been tasked with locating the customer who has the third highest total transaction amount.

The output should include the customer's id, as well as their first name and last name. For ranking the customers, use type of ranking with no gaps between subsequent ranks.

**Data View**  
customers

id: int64  
first_name: object  
last_name: object  
city: object  
address: object  
phone_number: object  

card_orders  

order_id: int64  
cust_id: int64  
order_date: datetime64[ns]  
order_details: object   
total_order_cost: int64  


In [ ]:
(
    pd.merge(
        customers[["id", "first_name", "last_name"]],
        card_orders.drop("order_id", axis=1)
        .groupby(["cust_id"])
        .sum("total_order_cost")
        .reset_index()
        .rename(columns={"cust_id": "id"}),
        how="right",
        on="id"["total_order_cost"].assign().rank(method="dense", ascending=False),
    )
)
